In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

session = cc.database.create_session()

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:45282 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
# input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [ ]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

mld_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='mld').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE))
print(mld_z0_1_mod_month)

mld_z0_1_mod_month.load()
print(mld_z0_1_mod_month)

xt_ocean = mld_z0_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
mld_z0_1_mod_month = mld_z0_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(mld_z0_1_mod_month)

mld_z0_1_mod = mld_z0_1_mod_month.groupby('time.year').mean('time')
print(mld_z0_1_mod)

mld_z0_1_mod = mld_z0_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(mld_z0_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'mld' (time: 3600, yt_ocean: 160, xt_ocean: 90)>
dask.array<getitem, shape=(3600, 160, 90), dtype=float32, chunksize=(1, 160, 90), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -269.5 -268.5 -267.5 ... -182.5 -181.5 -180.5
  * yt_ocean  (yt_ocean) float64 -69.63 -69.18 -68.71 ... 17.85 18.59 19.36
  * time      (time) object 1958-01-16 12:00:00 ... 2257-12-16 12:00:00
Attributes:
    long_name:      mixed layer depth determined by density criteria
    units:          m
    valid_range:    [      0. 1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  ocean_mixed_layer_thickness_defined_by_sigma_t


In [ ]:
year = mld_z0_1_mod.year
lat_1_mod = mld_z0_1_mod.yt_ocean
lon_1_mod = mld_z0_1_mod.xt_ocean

mld_z0_1_trans_mod = mld_z0_1_mod
print(np.shape(mld_z0_1_trans_mod))
mld_z0_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='mld_z0_1')
mld_z0_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='mld_z0_1')
mld_z0_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='mld_z0_1')
for iid, i in enumerate(lat_1_mod):
    for jid, j in enumerate(lon_1_mod):
        mld_z0_1_slope_mod[iid, jid], _, _, mld_z0_1_p_value_mod[iid, jid], mld_z0_1_std_err_mod[iid, jid] = \
        stats.linregress(year, mld_z0_1_trans_mod[:, iid, jid])
    print('lat_1_mod ' + str(np.array(i)))
print(mld_z0_1_slope_mod)

In [7]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

mld_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='mld').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE))
print(mld_z0_025_mod_month)

mld_z0_025_mod_month.load()
print(mld_z0_025_mod_month)

xt_ocean = mld_z0_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
mld_z0_025_mod_month = mld_z0_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(mld_z0_025_mod_month)

mld_z0_025_mod = mld_z0_025_mod_month.groupby('time.year').mean('time')
print(mld_z0_025_mod)

mld_z0_025_mod = mld_z0_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(mld_z0_025_mod)

025deg_jra55v13_iaf_gmredi6


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'mld' (time: 3600, yt_ocean: 475, xt_ocean: 360)>
dask.array<getitem, shape=(3600, 475, 360), dtype=float32, chunksize=(1, 435, 360), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -269.9 -269.6 -269.4 ... -180.6 -180.4 -180.1
  * yt_ocean  (yt_ocean) float64 -69.99 -69.88 -69.78 ... 19.49 19.72 19.96
  * time      (time) object 1958-01-14 12:00:00 ... 2257-12-14 12:00:00
Attributes:
    long_name:      mixed layer depth determined by density criteria
    units:          m
    valid_range:    [      0. 1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  ocean_mixed_layer_thickness_defined_by_sigma_t
<xarray.DataArray 'mld' (time: 3600, yt_ocean: 475, xt_ocean: 360)>
array([[[        nan,         nan,         nan, ...,   32.609997,
           31.959835,   31.626616],
        [        nan,         nan,         nan, ...,   32.621048,
           32.4

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'mld' (year: 300, yt_ocean: 475, xt_ocean: 360)>
array([[[         nan,          nan,          nan, ...,   92.62264  ,
           92.83471  ,   92.05656  ],
        [         nan,          nan,          nan, ...,   94.88196  ,
           94.81025  ,   94.02173  ],
        [         nan,          nan,          nan, ...,   96.34815  ,
           96.01772  ,   95.566414 ],
        ...,
        [  19.987198 ,   19.072927 ,   18.783594 , ...,   46.972397 ,
           44.574112 ,   44.293915 ],
        [  17.133347 ,   16.45027  ,   16.515009 , ...,   48.506817 ,
           47.35506  ,   43.569168 ],
        [  18.141832 ,   18.531965 ,   18.77233  , ...,   48.79459  ,
           48.615955 ,   48.020336 ]],

       [[         nan,          nan,          nan, ...,   93.993195 ,
           93.296295 ,   94.13793  ],
        [         nan,          nan,          nan, ...,   88.19041  ,
           87.511375 ,   87.64272  ],
        [         nan,          nan,          nan, ...

In [8]:
lat_025_mod = mld_z0_025_mod.yt_ocean
lon_025_mod = mld_z0_025_mod.xt_ocean

mld_z0_025_trans_mod = mld_z0_025_mod
print(np.shape(mld_z0_025_trans_mod))
mld_z0_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='mld_z0_025')
mld_z0_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='mld_z0_025')
mld_z0_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='mld_z0_025')
for iid, i in enumerate(lat_025_mod):
    for jid, j in enumerate(lon_025_mod):
        mld_z0_025_slope_mod[iid, jid], _, _, mld_z0_025_p_value_mod[iid, jid], mld_z0_025_std_err_mod[iid, jid] = \
        stats.linregress(year, mld_z0_025_trans_mod[:, iid, jid])
    print('lat_025_mod ' + str(np.array(i)))
print(mld_z0_025_slope_mod)

(14, 475, 360)
lat_025_mod -69.98914896732052
lat_025_mod -69.8835503781156
lat_025_mod -69.77795178891068
lat_025_mod -69.67235319970577
lat_025_mod -69.56675461050085
lat_025_mod -69.46115602129593
lat_025_mod -69.35555743209102
lat_025_mod -69.2499588428861
lat_025_mod -69.14436025368119
lat_025_mod -69.03876166447627
lat_025_mod -68.93316307527135
lat_025_mod -68.82756448606644
lat_025_mod -68.72196589686152
lat_025_mod -68.6163673076566
lat_025_mod -68.51076871845169
lat_025_mod -68.40517012924677
lat_025_mod -68.29957154004185
lat_025_mod -68.19397295083694
lat_025_mod -68.08837436163202
lat_025_mod -67.9827757724271
lat_025_mod -67.87717718322219
lat_025_mod -67.77157859401727
lat_025_mod -67.66598000481235
lat_025_mod -67.56038141560744
lat_025_mod -67.45478282640252
lat_025_mod -67.3491842371976
lat_025_mod -67.24358564799269
lat_025_mod -67.13798705878777
lat_025_mod -67.03238846958286
lat_025_mod -66.92678988037794
lat_025_mod -66.82119129117302
lat_025_mod -66.7155927019681

lat_025_mod -30.778202070190314
lat_025_mod -30.56317292803898
lat_025_mod -30.34766755148325
lat_025_mod -30.131685174432455
lat_025_mod -29.915230577717967
lat_025_mod -29.698303070212805
lat_025_mod -29.480907483686703
lat_025_mod -29.2630432019257
lat_025_mod -29.044715107989248
lat_025_mod -28.82592266049411
lat_025_mod -28.606670794106993
lat_025_mod -28.38695904216067
lat_025_mod -28.166792391212045
lat_025_mod -27.946170449161986
lat_025_mod -27.72509825470525
lat_025_mod -27.50357549012893
lat_025_mod -27.28160724647727
lat_025_mod -27.059193280207044
lat_025_mod -26.83633873488693
lat_025_mod -26.61304344089147
lat_025_mod -26.389312594451354
lat_025_mod -26.165146099570944
lat_025_mod -25.940549205242547
lat_025_mod -25.715521888775697
lat_025_mod -25.490069451985352
lat_025_mod -25.264191945124278
lat_025_mod -25.037894722851966
lat_025_mod -24.811177907964588
lat_025_mod -24.58404690794839
lat_025_mod -24.356501917704698
lat_025_mod -24.12854839748857
lat_025_mod -23.90018

In [8]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

mld_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='mld').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE))
print(mld_z0_01_mod_month)

mld_z0_01_mod_month.load()
print(mld_z0_01_mod_month)

xt_ocean = mld_z0_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
mld_z0_01_mod_month = mld_z0_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(mld_z0_01_mod_month)

mld_z0_01_mod = mld_z0_01_mod_month.groupby('time.year').mean('time')
print(mld_z0_01_mod)

mld_z0_01_mod = mld_z0_01_mod.sel(year=slice(2004,2017))
print(mld_z0_01_mod)

01deg_jra55v13_iaf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'mld' (time: 396, yt_ocean: 1186, xt_ocean: 900)>
dask.array<getitem, shape=(396, 1186, 900), dtype=float32, chunksize=(1, 675, 800), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -70.0 -69.96 -69.92 ... 19.75 19.84 19.94
  * xt_ocean  (xt_ocean) float64 -269.9 -269.8 -269.7 ... -180.2 -180.1 -180.0
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
Attributes:
    long_name:      mixed layer depth determined by density criteria
    units:          m
    valid_range:    [      0. 1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  ocean_mixed_layer_thickness_defined_by_sigma_t
<xarray.DataArray 'mld' (time: 396, yt_ocean: 1186, xt_ocean: 900)>
array([[[        nan,         nan,         nan, ...,  15.395936 ,
          15.585955 ,  15.509178 ],
        [        nan,         nan,         nan, ...,  15.075679 ,
          15.421

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'mld' (year: 33, yt_ocean: 1186, xt_ocean: 900)>
array([[[       nan,        nan,        nan, ..., 73.30273  ,
         73.75181  , 73.23481  ],
        [       nan,        nan,        nan, ..., 74.3841   ,
         74.842865 , 75.16219  ],
        [       nan,        nan,        nan, ..., 75.52942  ,
         75.35343  , 75.62504  ],
        ...,
        [17.513548 , 17.615759 , 17.856134 , ..., 45.019302 ,
         46.005123 , 43.766357 ],
        [17.379044 , 17.593456 , 17.357565 , ..., 43.90443  ,
         45.42072  , 44.17143  ],
        [17.31641  , 16.890162 , 15.779262 , ..., 42.68601  ,
         43.957928 , 44.576336 ]],

       [[       nan,        nan,        nan, ..., 60.667477 ,
         60.728546 , 60.37511  ],
        [       nan,        nan,        nan, ..., 61.196518 ,
         61.368633 , 61.19894  ],
        [       nan,        nan,        nan, ..., 61.68252  ,
         61.73088  , 61.851    ],
        ...,
        [13.494056 , 13.729698 , 13.90837

In [ ]:
year = mld_z0_01_mod.year
lat_01_mod = mld_z0_01_mod.yt_ocean
lon_01_mod = mld_z0_01_mod.xt_ocean

mld_z0_01_trans_mod = mld_z0_01_mod
print(np.shape(mld_z0_01_trans_mod))
mld_z0_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='mld_z0_01')
mld_z0_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='mld_z0_01')
mld_z0_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='mld_z0_01')
for iid, i in enumerate(lat_01_mod):
    for jid, j in enumerate(lon_01_mod):
        mld_z0_01_slope_mod[iid, jid], _, _, mld_z0_01_p_value_mod[iid, jid], mld_z0_01_std_err_mod[iid, jid] = \
        stats.linregress(year, mld_z0_01_trans_mod[:, iid, jid])
    print('lat_01_mod ' + str(np.array(i)))
print(mld_z0_01_slope_mod)

(14, 1186, 900)
lat_01_mod -69.99968147628742
lat_01_mod -69.95744212190694
lat_01_mod -69.91520276752645
lat_01_mod -69.87296341314597
lat_01_mod -69.83072405876548
lat_01_mod -69.788484704385
lat_01_mod -69.74624535000451
lat_01_mod -69.70400599562403
lat_01_mod -69.66176664124355
lat_01_mod -69.61952728686306
lat_01_mod -69.57728793248258
lat_01_mod -69.53504857810209
lat_01_mod -69.49280922372161
lat_01_mod -69.45056986934112
lat_01_mod -69.40833051496064
lat_01_mod -69.36609116058015
lat_01_mod -69.32385180619967
lat_01_mod -69.28161245181919
lat_01_mod -69.2393730974387
lat_01_mod -69.19713374305822
lat_01_mod -69.15489438867773
lat_01_mod -69.11265503429725
lat_01_mod -69.07041567991676
lat_01_mod -69.02817632553628
lat_01_mod -68.9859369711558
lat_01_mod -68.94369761677531
lat_01_mod -68.90145826239483
lat_01_mod -68.85921890801434
lat_01_mod -68.81697955363386
lat_01_mod -68.77474019925337
lat_01_mod -68.73250084487289
lat_01_mod -68.6902614904924
lat_01_mod -68.64802213611192

In [ ]:
# mld_z0_1_mod_xr = xr.DataArray(mld_z0_1_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='mld_z0_1_mod',
#                         coords=[lat_1_mod, lon_1_mod, year], 
#                         dims=['yt_ocean', 'xt_ocean', 'year'])
# print(mld_z0_1_mod_xr)
# mld_z0_1_mod_xr.to_netcdf(output_path + 'mld_z0_1_mod.nc')

# mld_z0_1_slope_mod_xr = xr.DataArray(mld_z0_1_slope_mod, name='mld_z0_1_slope_mod',
#                         coords=[lat_1_mod, lon_1_mod], 
#                         dims=['yt_ocean', 'xt_ocean'])
# print(mld_z0_1_slope_mod_xr)
# mld_z0_1_slope_mod_xr.to_netcdf(output_path + 'mld_z0_1_slope_mod.nc')

# mld_z0_1_p_value_mod_xr = xr.DataArray(mld_z0_1_p_value_mod, name='mld_z0_1_p_value_mod',
#                         coords=[lat_1_mod, lon_1_mod], 
#                         dims=['yt_ocean', 'xt_ocean'])
# print(mld_z0_1_p_value_mod_xr)
# mld_z0_1_p_value_mod_xr.to_netcdf(output_path + 'mld_z0_1_p_value_mod.nc')

# mld_z0_1_std_err_mod_xr = xr.DataArray(mld_z0_1_std_err_mod, name='mld_z0_1_std_err_mod',
#                         coords=[lat_1_mod, lon_1_mod], 
#                         dims=['yt_ocean', 'xt_ocean'])
# print(mld_z0_1_std_err_mod_xr)
# mld_z0_1_std_err_mod_xr.to_netcdf(output_path + 'mld_z0_1_std_err_mod.nc')



# mld_z0_025_mod_xr = xr.DataArray(mld_z0_025_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='mld_z0_025_mod',
#                         coords=[lat_025_mod, lon_025_mod, year], 
#                         dims=['yt_ocean', 'xt_ocean', 'year'])
# print(mld_z0_025_mod_xr)
# mld_z0_025_mod_xr.to_netcdf(output_path + 'mld_z0_025_mod.nc')

# mld_z0_025_slope_mod_xr = xr.DataArray(mld_z0_025_slope_mod, name='mld_z0_025_slope_mod',
#                         coords=[lat_025_mod, lon_025_mod], 
#                         dims=['yt_ocean', 'xt_ocean'])
# print(mld_z0_025_slope_mod_xr)
# mld_z0_025_slope_mod_xr.to_netcdf(output_path + 'mld_z0_025_slope_mod.nc')

# mld_z0_025_p_value_mod_xr = xr.DataArray(mld_z0_025_p_value_mod, name='mld_z0_025_p_value_mod',
#                         coords=[lat_025_mod, lon_025_mod], 
#                         dims=['yt_ocean', 'xt_ocean'])
# print(mld_z0_025_p_value_mod_xr)
# mld_z0_025_p_value_mod_xr.to_netcdf(output_path + 'mld_z0_025_p_value_mod.nc')

# mld_z0_025_std_err_mod_xr = xr.DataArray(mld_z0_025_std_err_mod, name='mld_z0_025_std_err_mod',
#                         coords=[lat_025_mod, lon_025_mod], 
#                         dims=['yt_ocean', 'xt_ocean'])
# print(mld_z0_025_std_err_mod_xr)
# mld_z0_025_std_err_mod_xr.to_netcdf(output_path + 'mld_z0_025_std_err_mod.nc')



mld_z0_01_mod_xr = xr.DataArray(mld_z0_01_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='mld_z0_01_mod',
                        coords=[lat_01_mod, lon_01_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(mld_z0_01_mod_xr)
mld_z0_01_mod_xr.to_netcdf(output_path + 'mld_z0_01_mod.nc')

mld_z0_01_slope_mod_xr = xr.DataArray(mld_z0_01_slope_mod, name='mld_z0_01_slope_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(mld_z0_01_slope_mod_xr)
mld_z0_01_slope_mod_xr.to_netcdf(output_path + 'mld_z0_01_slope_mod.nc')

mld_z0_01_p_value_mod_xr = xr.DataArray(mld_z0_01_p_value_mod, name='mld_z0_01_p_value_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(mld_z0_01_p_value_mod_xr)
mld_z0_01_p_value_mod_xr.to_netcdf(output_path + 'mld_z0_01_p_value_mod.nc')

mld_z0_01_std_err_mod_xr = xr.DataArray(mld_z0_01_std_err_mod, name='mld_z0_01_std_err_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(mld_z0_01_std_err_mod_xr)
mld_z0_01_std_err_mod_xr.to_netcdf(output_path + 'mld_z0_01_std_err_mod.nc')